In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:

# Paths
TRAIN_PATH = "C:/Users/HP/Downloads/pests dataset/train"
TEST_PATH = "C:/Users/HP/Downloads/pests dataset/test"

# Data generators with stronger augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.25,
    height_shift_range=0.25,
    brightness_range=[0.6, 1.4],
    zoom_range=0.4,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets with increased image size
train_generator = train_datagen.flow_from_directory(
  TRAIN_PATH,
    target_size=(160, 160),  
    batch_size=32,
    class_mode='categorical'
)

valid_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(160, 160),  
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH, target_size=(160, 160), batch_size=32, class_mode='categorical', shuffle=False
)



Found 6082 images belonging to 18 classes.
Found 6082 images belonging to 18 classes.
Found 1266 images belonging to 18 classes.


In [ ]:
# Base model with partial fine-tuning
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3), include_top=False, weights='imagenet'
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1),
    ModelCheckpoint("pest_detection_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
]


history = model.fit(
    train_generator,
    epochs=30,
    validation_data=valid_generator,
    callbacks=callbacks
)



Epoch 1/30
191/191 [==============================] - ETA: 0s - loss: 2.3052 - accuracy: 0.3321
Epoch 1: val_accuracy improved from -inf to 0.57547, saving model to pest_detection_model.h5
191/191 [==============================] - 226s 1s/step - loss: 2.3052 - accuracy: 0.3321 - val_loss: 1.5711 - val_accuracy: 0.5755 - lr: 1.0000e-04
Epoch 2/30
191/191 [==============================] - ETA: 0s - loss: 1.5267 - accuracy: 0.5572
Epoch 2: val_accuracy improved from 0.57547 to 0.65209, saving model to pest_detection_model.h5
191/191 [==============================] - 165s 865ms/step - loss: 1.5267 - accuracy: 0.5572 - val_loss: 1.2164 - val_accuracy: 0.6521 - lr: 1.0000e-04
Epoch 3/30
191/191 [==============================] - ETA: 0s - loss: 1.2847 - accuracy: 0.6146
Epoch 3: val_accuracy improved from 0.65209 to 0.68777, saving model to pest_detection_model.h5
191/191 [==============================] - 168s 881ms/step - loss: 1.2847 - accuracy: 0.6146 - val_loss: 1.0613 - val_accuracy

In [5]:

base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


history_fine = model.fit(
    train_generator,
    epochs=30,
    validation_data=valid_generator,
    callbacks=callbacks
)

Epoch 1/30
191/191 [==============================] - ETA: 0s - loss: 1.0019 - accuracy: 0.6940
Epoch 1: val_accuracy did not improve from 0.80944
191/191 [==============================] - 315s 2s/step - loss: 1.0019 - accuracy: 0.6940 - val_loss: 0.6266 - val_accuracy: 0.7999 - lr: 1.0000e-05
Epoch 2/30
191/191 [==============================] - ETA: 0s - loss: 0.8723 - accuracy: 0.7261
Epoch 2: val_accuracy did not improve from 0.80944
191/191 [==============================] - 298s 2s/step - loss: 0.8723 - accuracy: 0.7261 - val_loss: 0.6131 - val_accuracy: 0.8006 - lr: 1.0000e-05
Epoch 3/30
191/191 [==============================] - ETA: 0s - loss: 0.8365 - accuracy: 0.7391
Epoch 3: val_accuracy did not improve from 0.80944
191/191 [==============================] - 302s 2s/step - loss: 0.8365 - accuracy: 0.7391 - val_loss: 0.6130 - val_accuracy: 0.8091 - lr: 1.0000e-05
Epoch 4/30
191/191 [==============================] - ETA: 0s - loss: 0.8115 - accuracy: 0.7458
Epoch 4: val_acc

In [6]:

loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc*100:.2f}%")

predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

40/40 [==============================] - 11s 278ms/step - loss: 0.5689 - accuracy: 0.8175
Test Accuracy: 81.75%
40/40 [==============================] - 11s 244ms/step
                 precision    recall  f1-score   support

          Aphid       0.44      0.35      0.39        20
Colorado beetle       0.85      0.96      0.90        47
    Gryllotalpa       0.63      0.93      0.75        28
       Mealybug       0.45      1.00      0.62        37
       Whitefly       0.86      0.25      0.39        24
       Wireworm       0.58      1.00      0.73        11
           ants       0.83      0.97      0.89        99
           bees       0.88      0.96      0.92        95
         beetle       0.68      0.53      0.60        85
   catterpillar       0.90      0.63      0.74       105
     earthworms       0.84      0.69      0.76        77
         earwig       0.75      0.61      0.67        76
    grasshopper       0.80      0.74      0.77        95
           moth       0.94      0

In [7]:
from tensorflow.keras.models import load_model

model = load_model("pest_detection_model.h5")
model.save("pest_detection_model_float32", save_format="tf")


INFO:tensorflow:Assets written to: pest_detection_model_float32\assets


INFO:tensorflow:Assets written to: pest_detection_model_float32\assets


In [ ]:


model.save("pest_detection_model_saved")


converter = tf.lite.TFLiteConverter.from_saved_model("pest_detection_model_saved")
converter.target_spec.supported_types = [tf.float32] 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("pest_detection_model.tflite", "wb") as f:
    f.write(tflite_model)




INFO:tensorflow:Assets written to: pest_detection_model_saved\assets


INFO:tensorflow:Assets written to: pest_detection_model_saved\assets


In [12]:
import os

TRAIN_PATH = "C:/Users/HP/Downloads/pests dataset/train"
class_names = sorted(os.listdir(TRAIN_PATH))

with open(" PEST label.txt", "w") as f:
    for class_name in class_names:
        f.write(class_name + "\n")

print("SAVED")


SAVED
